<a href="https://colab.research.google.com/github/SharmaSanskar/ml-story-generation/blob/main/story_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# STORY GENERATION USING GRUs
It is difficult to learn long term dependencies using vanilla RNNs due to the problem of vanishing and exploding gradients. LSTMs and GRUs solve this problem by maintaining a cell state and controlling the cell state using a set of gates.

### Importing Libraries

In [ ]:
!pip install gradio

     |████████████████████████████████| 871 kB 10.6 MB/s 
     |████████████████████████████████| 52 kB 890 kB/s 
     |████████████████████████████████| 52 kB 1.5 MB/s 
     |████████████████████████████████| 210 kB 50.2 MB/s 
     |████████████████████████████████| 2.0 MB 51.7 MB/s 
     |████████████████████████████████| 1.1 MB 49.3 MB/s 
     |████████████████████████████████| 271 kB 44.1 MB/s 
     |████████████████████████████████| 144 kB 47.5 MB/s 
     |████████████████████████████████| 94 kB 3.8 MB/s 
     |████████████████████████████████| 10.9 MB 45.7 MB/s 
     |████████████████████████████████| 58 kB 6.3 MB/s 
     |████████████████████████████████| 79 kB 8.8 MB/s 
     |████████████████████████████████| 856 kB 49.0 MB/s 
     |████████████████████████████████| 3.6 MB 48.3 MB/s 
     |████████████████████████████████| 61 kB 406 kB/s 
     |████████████████████████████████| 58 kB 5.8 MB/s 
  Created wheel for ffmpy: filename=ffmpy-0.3.0-py3-none-any.whl size=4712 sha256=e96

In [ ]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.layers import Embedding, GRU, Dense, Dropout
from keras.models import Sequential
import tensorflow.keras.utils as ku

In [ ]:
# Input file
filepath = './harry_potter.txt'
text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()

In [ ]:
text

'harry potter and the sorcerer\'s stone\n\n\nchapter one\n\nthe boy who lived\n\nmr. and mrs. dursley, of number four, privet drive, were proud to say\nthat they were perfectly normal, thank you very much. they were the last\npeople you\'d expect to be involved in anything strange or mysterious,\nbecause they just didn\'t hold with such nonsense.\n\nmr. dursley was the director of a firm called grunnings, which made\ndrills. he was a big, beefy man with hardly any neck, although he did\nhave a very large mustache. mrs. dursley was thin and blonde and had\nnearly twice the usual amount of neck, which came in very useful as she\nspent so much of her time craning over garden fences, spying on the\nneighbors. the dursleys had a small son called dudley and in their\nopinion there was no finer boy anywhere.\n\nthe dursleys had everything they wanted, but they also had a secret, and\ntheir greatest fear was that somebody would discover it. they didn\'t\nthink they could bear it if anyone foun

### Cleaning the Text
Splitting each line and removing the punctuations.

In [ ]:
import string
def clean_text(line):
  line = "".join(v for v in line if v not in string.punctuation)
  return line

In [ ]:
corpus = [clean_text(line) for line in text.split("\n") if line.strip()]

In [ ]:
corpus[:10]

['harry potter and the sorcerers stone',
 'chapter one',
 'the boy who lived',
 'mr and mrs dursley of number four privet drive were proud to say',
 'that they were perfectly normal thank you very much they were the last',
 'people youd expect to be involved in anything strange or mysterious',
 'because they just didnt hold with such nonsense',
 'mr dursley was the director of a firm called grunnings which made',
 'drills he was a big beefy man with hardly any neck although he did',
 'have a very large mustache mrs dursley was thin and blonde and had']

# Tokenization
Extracting the tokens and their indices from each sentence. Since ML models only understand numbers. 

In [ ]:
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    # Tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    # Convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

In [ ]:
inp_sequences, total_words = get_sequence_of_tokens(corpus)
inp_sequences[:10]

[[7, 123],
 [7, 123, 2],
 [7, 123, 2, 1],
 [7, 123, 2, 1, 631],
 [7, 123, 2, 1, 631, 155],
 [607, 38],
 [1, 140],
 [1, 140, 73],
 [1, 140, 73, 1036],
 [144, 2]]

In [ ]:
total_words

6036

### Padding
Each sequence of tokens is padded to make their length equal.

In [ ]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

### Creating GRU Model
GRUs only have 2 gates (Update and Reset) compared to the 3 gates in LSTMs. This makes GRUs faster in terms of training speed. Also GRUs outperform LSTMs in scenario of long text and small dataset.

In [ ]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()   
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))  
    # Add Hidden Layer 1 - GRU Layer
    model.add(GRU(256, return_sequences=True))
    model.add(Dropout(0.2))
    # Add Hidden Layer 2 - GRU Layer
    model.add(GRU(256))
    model.add(Dropout(0.2))
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')  
    return model

model = create_model(max_sequence_len, total_words)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 10)            60360     
                                                                 
 gru (GRU)                   (None, 17, 256)           205824    
                                                                 
 dropout (Dropout)           (None, 17, 256)           0         
                                                                 
 gru_1 (GRU)                 (None, 256)               394752    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense (Dense)               (None, 6036)              1551252   
                                                                 
Total params: 2,212,188
Trainable params: 2,212,188
Non-

In [ ]:
%%time
model.fit(predictors, label, epochs=200, verbose=1)

Epoch 1/200
2187/2187 [==============================] - 49s 19ms/step - loss: 6.9547
Epoch 2/200
2187/2187 [==============================] - 40s 19ms/step - loss: 6.7350
Epoch 3/200
2187/2187 [==============================] - 40s 18ms/step - loss: 6.6340
Epoch 4/200
2187/2187 [==============================] - 40s 18ms/step - loss: 6.5705
Epoch 5/200
2187/2187 [==============================] - 40s 18ms/step - loss: 6.5175
Epoch 6/200
2187/2187 [==============================] - 40s 18ms/step - loss: 6.4585
Epoch 7/200
2187/2187 [==============================] - 39s 18ms/step - loss: 6.4154
Epoch 8/200
2187/2187 [==============================] - 39s 18ms/step - loss: 6.3582
Epoch 9/200
2187/2187 [==============================] - 38s 17ms/step - loss: 6.3132
Epoch 10/200
2187/2187 [==============================] - 38s 17ms/step - loss: 6.2795
Epoch 11/200
2187/2187 [==============================] - 39s 18ms/step - loss: 6.2444
Epoch 12/200
2187/2187 [============================

In [ ]:
model.save("/content/drive/MyDrive/story_generator_200")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/story_generator_200/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/story_generator_200/assets


### Generate text
The prompt is tokenized, padded and passed to the model. The word with maximum probability is selected and appended to the output.

In [ ]:
def generate_text(seed_text, next_words):
    for _ in range(int(next_words)):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict(token_list)
        predicted_class = np.argmax(predicted, axis=1)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted_class:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

In [ ]:
import gradio as gr
gr.Interface(generate_text, title="GRU Text Generator", inputs=["text", "number"], outputs="textbox").launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://55391.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<fastapi.applications.FastAPI at 0x7f2eebf7d510>,
 'http://127.0.0.1:7860/',
 'https://55391.gradio.app')

----

# RISE OF TRANSFORMERS
In 2017, Google AI released Transformer architecture. It was able to process sentences as a whole and focus on certain important words. Many state-of-the-art NLP models today use Transformer architecture.

### Importing Libraries

In [ ]:
!pip install simpletransformers

     |████████████████████████████████| 248 kB 10.8 MB/s 
     |████████████████████████████████| 3.5 MB 15.6 MB/s 
     |████████████████████████████████| 6.8 MB 44.5 MB/s 
     |████████████████████████████████| 1.7 MB 43.4 MB/s 
     |████████████████████████████████| 311 kB 48.5 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 9.7 MB 18.7 MB/s 
     |████████████████████████████████| 1.2 MB 45.2 MB/s 
     |████████████████████████████████| 67 kB 5.0 MB/s 
     |████████████████████████████████| 895 kB 40.9 MB/s 
     |████████████████████████████████| 596 kB 44.1 MB/s 
     |████████████████████████████████| 180 kB 51.8 MB/s 
     |████████████████████████████████| 143 kB 53.9 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 133 kB 56.1 MB/s 
     |████████████████████████████████| 243 kB 47.3 MB/s 
     |████████████████████████████████| 4.3 MB 23.2 MB/s 
     |██████████████

In [ ]:
import logging
from simpletransformers.language_modeling import LanguageModelingModel, LanguageModelingArgs
from simpletransformers.language_generation import LanguageGenerationModel, LanguageGenerationArgs

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

### Fine-tuning GPT-2
GPT-2 is a large transformer-based language model with 1.5 billion parameters, trained on a dataset of 8 million web pages. <br/>
Fine-tuning GPT-2 model on input text will allow us to take advantage of what the model has already learned without having to develop it from scratch. 

In [ ]:
model_args = LanguageModelingArgs()
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.num_train_epochs = 3
model_args.dataset_type = "simple"
model_args.mlm = False

# Input file
train_file = "harry_potter.txt"

model = LanguageModelingModel(
    "gpt2", "gpt2", args=model_args, train_files=train_file
)

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

In [ ]:
%%time
model.train_model(train_file)

  0%|          | 0/7646 [00:00<?, ?it/s]

  0%|          | 0/841 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/106 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running Epoch 1 of 3:   0%|          | 0/106 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/106 [00:00<?, ?it/s]

CPU times: user 4min 7s, sys: 1min 56s, total: 6min 3s
Wall time: 6min 28s


(318, 3.5872417253518254)

### Generate Text

In [ ]:
def gpt_generate(seed_text, next_words):
  language_args = LanguageGenerationArgs()
  language_args.max_length = int(next_words)
  model = LanguageGenerationModel("gpt2", "outputs/checkpoint-318-epoch-3", args=language_args)
  output = model.generate(seed_text)
  return output[0]

In [ ]:
gr.Interface(gpt_generate, title="GPT-2 Text Generator", inputs=["text", "number"], outputs="textbox").launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://25182.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<fastapi.applications.FastAPI at 0x7f2eebf7d510>,
 'http://127.0.0.1:7861/',
 'https://25182.gradio.app')